# Integrate Dutch Ministers (DM) in CLERUS

The process of integrating the DRC dataset into CLERUS has been extensively described in [1_1_DRC_1555-1816.ipynb](1_1_DRC_1555-1816.ipynb). In this notebook it has also been described that for the final integration of DRC into CLERUS a curation steps was made. In [1_2_DM_1572-2004.ipynb](1_2_DM_1572-2004.ipynb) the process is presented how to extract individuals from the DM dataset can be extracted. In this notebook these individuals are connected with the individuals from the curated DRC dataset. 

For this, the strategy is to extend the DM dataset with the clerus_id for those fields that are present in the drc dataset. By doing this in this way, the DM dataset stays as much intact as possible. Once the DM is extended with the clerus_id for records that match, the DM dataset will be parsed to clerus.


In [1]:
# import the required libraries
import pandas as pd

In [2]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )
folderlink = '..//data//'
input_folder = 'input//'
output_folder = 'output//'


In [4]:
# Set links to the files
d1_csv = folderlink+input_folder+"01_clerus_bio_curated.csv"
d11_csv = folderlink+input_folder+"12_clerus_role_curated.csv"
d22_csv = folderlink+output_folder+"12_roles_dm.csv"

In [9]:
# Import the csvs as dataframes
d1_col_data_type = {'clerus_id': pd.Int64Dtype()}
d1 = pd.read_csv(d1_csv, sep=';', dtype=d1_col_data_type, encoding='utf-8')

d11_col_data_type = {'clerus_id': pd.Int64Dtype(), 'role_start_year': pd.Int64Dtype(), 'role_end_year': pd.Int64Dtype()}
d11 = pd.read_csv(d11_csv, sep=';', dtype=d11_col_data_type, encoding='utf-8')

d22_data_types = {'jaar vertrek': pd.Int64Dtype(), 'ind_id': pd.Int64Dtype(),'dag intrede': pd.Int64Dtype(), 'dag vertrek': pd.Int64Dtype() }
d22 = pd.read_csv(d22_csv, dtype=d22_data_types)


In [10]:
# from the 01_clerus_bio table we only require the name of the individual to be linked with DM
d1_name = d1[['clerus_id', 'first_name', 'infix', 'surname']]


In [11]:
d1_name.head()

,clerus_id,first_name,infix,surname
0,1657,Johannes,NaN,Acronius
1,20035,Laurentius,NaN,Alberti
2,20036,W.,NaN,Alberts
3,20037,Claes,NaN,Albertsz.
4,20038,Albertus,NaN,NaN


In [12]:
d1_d11_merge = pd.merge(d1_name, d11, on='clerus_id', how='inner')

In [13]:
# filter out all role_type that are not minister (predikant)
d1_d11_merge['role_type'] = d1_d11_merge['role_type'].fillna('')
d1_d11 = d1_d11_merge[d1_d11_merge['role_type'].str.contains('pred', case=False)]


In [14]:
# The first stategy to link DM with DRC is to link them based on the names. In DM the name is stored in one field (i.e. predikant). Since that field uses multiple methods to distinguish the first name, inifix and surname (e.g. . and ; etc.) we decided to map DRC to DM and remove all spaces and seperating characters to create a field that can be linked.

d1_d11['surname'] = d1_d11['surname'].fillna('')
d1_d11['first_name'] = d1_d11['first_name'].fillna('')
d1_d11['infix'] = d1_d11['infix'].fillna('')
d1_d11['join_name'] = d1_d11['surname']+d1_d11['first_name']+d1_d11['infix']


C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\347723106.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1_d11['surname'] = d1_d11['surname'].fillna('')
C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\347723106.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1_d11['first_name'] = d1_d11['first_name'].fillna('')
C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\347723106.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [15]:
# Here we clean the names of the ministers from DM from spaces and seperating characters
d22['d22join_name'] = d22['predikant'].str.replace(";","").str.replace(" ","").str.replace("  ","").str.replace(".","")
# We select the first string in the placename
d22['d22gemeente'] = d22['gemeente'].str.split().str[0]
# Add the start year
d22['d22jaar intrede'] = d22['jaar intrede'].astype(str).str.replace(";","").str.replace("","").str.replace("  ","")

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\3262996418.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  d22['d22join_name'] = d22['predikant'].str.replace(";","").str.replace(" ","").str.replace("  ","").str.replace(".","")


In [17]:
# Here we clean the joined names of the ministers from DRC from spaces and seperating characters
d1_d11['d11join_name'] = d1_d11['join_name'].str.replace(";","").str.replace(" ","").str.replace("  ","")

# We select the first word form the placename, since sometimes alternative names are included between brackets
d1_d11['d11_role_place']= d1_d11['role_place'].str.split().str[0]

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\3221726576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1_d11['d11join_name'] = d1_d11['join_name'].str.replace(";","").str.replace(" ","").str.replace("  ","")
C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\3221726576.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1_d11['d11_role_place']= d1_d11['role_place'].str.split().str[0]


In [18]:
# We create a joining field, which is a combination between the name, placename and start year
d1_d11['d11j_place_year']=d1_d11['d11join_name'].astype(str)+"_"+d1_d11['d11_role_place'].astype(str)+"_"+d1_d11['role_start_year'].astype(str)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\1443844824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1_d11['d11j_place_year']=d1_d11['d11join_name'].astype(str)+"_"+d1_d11['d11_role_place'].astype(str)+"_"+d1_d11['role_start_year'].astype(str)


In [19]:
# For dm a join field is created in a similar way after which the two are joined
d22['d22j_place_year']=d22['d22join_name'].astype(str)+"_"+d22['d22gemeente'].astype(str)+"_"+d22['d22jaar intrede'].astype(str)
d11_d22 = pd.merge(d1_d11, d22, left_on='d11j_place_year', right_on='d22j_place_year', how='inner')

In [20]:
# For dm a join field is created in a similar way after which the two are joined
id_counts_d11 = d1_d11['clerus_id'].value_counts()

In [21]:
# Since an individual form drc could have more minister roles in dm we are going to check for that. In d1_d11 the number of times an individual was minister is counted.
d1_d11['d11_count'] = d1_d11['clerus_id'].map(id_counts_d11)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\1249576984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1_d11['d11_count'] = d1_d11['clerus_id'].map(id_counts_d11)


In [22]:
# Next, a string is created combined with the clerus id. For DM we will do the same lateron
d1_d11['d11_count_unique'] = d1_d11['clerus_id'].astype(str)+"__"+d1_d11['d11_count'].astype(str)

C:\Users\MauricedeKleijn\AppData\Local\Temp\ipykernel_31304\2161369368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1_d11['d11_count_unique'] = d1_d11['clerus_id'].astype(str)+"__"+d1_d11['d11_count'].astype(str)


In [23]:
# The number of times the clerus_id is present in the joined tables
id_counts_d22_d11 = d11_d22['clerus_id'].value_counts()
d11_d22['d11_d22_count'] = d11_d22['clerus_id'].map(id_counts_d22_d11)

In [24]:
d11_d22['d11_d22_count_unique']= d11_d22['clerus_id'].astype(str)+"__"+d11_d22['d11_d22_count'].astype(str)

In [25]:
# join the two do see which ones are the same
countd11_countd11_d22 = pd.merge(d1_d11, d11_d22, left_on='d11_count_unique', right_on='d11_d22_count_unique', how='inner')

In [26]:
output_table = countd11_countd11_d22[['clerus_id_x','d11_count_unique','d11_d22_count_unique']].copy()

In [27]:
# remove duplicates
output_without_duplicates = output_table.drop_duplicates()

In [33]:
output_without_duplicates.head()

,clerus_id_x,d11_count_unique,d11_d22_count_unique
0,20042,20042__1,20042__1
1,20046,20046__1,20046__1
2,20394,20394__3,20394__3
11,20049,20049__1,20049__1
12,1,1__1,1__1


In [29]:
output_without_duplicates.to_csv(folderlink+output_folder+'ouput_nodup_first_word.csv', sep=';', encoding='utf-8', index=False)

In [41]:
output_without_duplicates.describe()

,clerus_id_x
count,7798.000000
mean,6605.392280
std,4941.647551
min,1.000000
25%,2957.250000
50%,5926.500000
75%,9035.750000
max,30020.000000


In [39]:
d11_d22_v2 = pd.merge(d11_d22, output_without_duplicates, left_on='clerus_id', right_on='clerus_id_x', how='left')



In [40]:
d11_d22_v2.head(100)

,clerus_id,first_name,infix,surname,role_type,role_place,role_place_id,role_classis_code,role_classis,role_parish,role_province,role_region,role_start_year,role_start_date_exact,role_start_year_accuracy,role_end_year,role_end_date_exact,role_end_year_accuracy,role_residence_place,role_residence_place_id,role_remarks,join_name,d11join_name,d11_role_place,d11j_place_year,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,individual_id,d22join_name,d22gemeente,d22jaar intrede,d22j_place_year,d11_d22_count,d11_d22_count_unique_x,clerus_id_x,d11_count_unique,d11_d22_count_unique_y
0,20041,Andreas,,Alhardi,predikant,Eenrum,NaN,NaN,NaN,NaN,NaN,NaN,1595,NaN,NaN,1606,NaN,NaN,NaN,NaN,"Eenrum juli 1595, neergelegd wegens twisten 1606",AlhardiAndreas,AlhardiAndreas,Eenrum,AlhardiAndreas_Eenrum_1595,11440,<NA>,GR,De Marne,Eenrum,NaN,Alhardi; Andreas,NaN,<NA>,NaN,1595,NaN,<NA>,NaN,1606,NaN,233,AlhardiAndreas,Eenrum,1595,AlhardiAndreas_Eenrum_1595,2,20041__2,<NA>,NaN,NaN
1,20041,Andreas,,Alhardi,predikant,Noordwolde (Gr),NaN,NaN,NaN,NaN,NaN,NaN,1607,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,Noordwolde (Gr) 1607,AlhardiAndreas,AlhardiAndreas,Noordwolde,AlhardiAndreas_Noordwolde_1607,31861,<NA>,GR,Middelstum,Noordwolde,NaN,Alhardi; Andreas,NaN,<NA>,NaN,1607,NaN,<NA>,NaN,1608,NaN,234,AlhardiAndreas,Noordwolde,1607,AlhardiAndreas_Noordwolde_1607,2,20041__2,<NA>,NaN,NaN
2,20042,Albertus,,Allardi,predikant,Goedereede,NaN,NaN,NaN,NaN,NaN,NaN,1574,NaN,NaN,1577,NaN,NaN,NaN,NaN,"Goedereede april 1574, afgezet wegens levenswandel juni 1577.",AllardiAlbertus,AllardiAlbertus,Goedereede,AllardiAlbertus_Goedereede_1574,14485,<NA>,ZH,Voorne-Putten,Goedereede,NaN,Allardi; Albertus,NaN,<NA>,NaN,1574,NaN,<NA>,NaN,1577,NaN,238,AllardiAlbertus,Goedereede,1574,AllardiAlbertus_Goedereede_1574,1,20042__1,20042,20042__1,20042__1
3,20043,Godefridus,,Allendorp,predikant,Groote Lindt en Heer Oudelandsambacht,NaN,NaN,NaN,NaN,NaN,NaN,1585,NaN,NaN,1592,NaN,NaN,NaN,NaN,Groote Lindt en Heer Oudelandsambacht 1585 tot 1592,AllendorpGodefridus,AllendorpGodefridus,Groote,AllendorpGodefridus_Groote_1585,16388,<NA>,ZH,Dordrecht,Groote Lindt,NaN,Allendorp; Godefridus,NaN,<NA>,NaN,1585,NaN,<NA>,NaN,1592,NaN,244,AllendorpGodefridus,Groote,1585,AllendorpGodefridus_Groote_1585,2,20043__2,<NA>,NaN,NaN
4,20043,Godefridus,,Allendorp,predikant,Papendrecht,NaN,NaN,NaN,NaN,NaN,NaN,1593,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,Papendrecht 1593,AllendorpGodefridus,AllendorpGodefridus,Papendrecht,AllendorpGodefridus_Papendrecht_1593,36153,<NA>,ZH,Dordrecht,Papendrecht,NaN,Allendorp; Godefridus,NaN,<NA>,NaN,1593,NaN,<NA>,NaN,1606,NaN,245,AllendorpGodefridus,Papendrecht,1593,AllendorpGodefridus_Papendrecht_1593,2,20043__2,<NA>,NaN,NaN
5,9001,Joannes,,Sluiter,hulppredikant,Raalte,NaN,NaN,NaN,NaN,NaN,NaN,1687,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,Raalte [H] 16 okt. 1687,SluiterJoannes,SluiterJoannes,Raalte,SluiterJoannes_Raalte_1687,37079,<NA>,OV,Zwolle,Raalte,NaN,Sluiter; Joannes,NaN,<NA>,NaN,1687,NaN,<NA>,NaN,1707,NaN,11194,SluiterJoannes,Raalte,1687,SluiterJoannes_Raalte_1687,2,9001__2,<NA>,NaN,NaN
6,9001,Joannes,,Sluiter,predikant,Steenwijk,NaN,NaN,NaN,NaN,NaN,NaN,1707,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,Steenwijk juli 1707,SluiterJoannes,SluiterJoannes,Steenwijk,SluiterJoannes_Steenwijk_1707,42722,<NA>,OV,Steenwijk,Steenwijk,NaN,Sluiter; Joannes,NaN,<NA>,NaN,1707,NaN,<NA>,NaN,1742,NaN,11194,SluiterJoannes,Steenwijk,1707,SluiterJoannes_Steenwijk_1707,2,9001__2,<NA>,NaN,NaN
7,20046,Johannes,,Altenhovius,predikant,Ouderkerk aan de Amstel,NaN,NaN,NaN,NaN,NaN,NaN,1596,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,Ouderkerk aan de Amstel 1596,AltenhoviusJohannes,AltenhoviusJohannes,Ouderkerk,AltenhoviusJohannes_Ouderkerk_1596,35459,<NA>,NH,Amsterdam,Ouderkerk aan de Amstel,NaN,Altenhovius; Johannes,NaN,<NA>,NaN,1596,NaN,<NA>,NaN,1616,NaN,287,AltenhoviusJohannes,Ouderkerk,1596,AltenhoviusJohannes_Ouderkerk_1596,1,20046__1,20046,20